### Retail Sales and Inventories

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Main Chart

In [2]:
# Retail sales
url = ('https://api.census.gov/data/timeseries/eits/marts/?'+
       'get=cell_value,time_slot_id,category_code&'+
       f'key={census_key}&data_type_code=SM&time=from+1992&'+
       'for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = ((pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
         .set_index(['date', 'category_code']))['cell_value']
         .unstack().astype('float'))
df.to_csv(data_dir / 'rs_raw.csv', index_label='date')

In [3]:
df = pd.read_csv(data_dir / 'rs_raw.csv', index_col='date', 
                 parse_dates=True)[['44000', '44X72', '44W72', '454']]
data = (df.pct_change(12) * 100).dropna()
data['NS_3M'] = data['454'].rolling(3).mean()
data.to_csv(data_dir / 'marts.csv', index_label='date', 
            float_format='%g')
nsgr = data['454'].iloc[-1]
totgr = data['44X72'].iloc[-1]
date_ns = None if nsgr <= totgr else 'm'
offset_ns = 0 if nsgr <= totgr else 0.4
date_tot = None if totgr < nsgr else 'm'
offset_tot = 0 if totgr < nsgr else 0.4
diff = nsgr - totgr
if abs(diff) < 5:
    offset_ns = offset_ns*2 - 0.15
    offset_tot = offset_tot*2 - 0.15

node = end_node(data['NS_3M'], 'blue!70!black', percent=True, 
                date=date_ns, offset=offset_ns, full_year=True)
write_txt(text_dir / 'rs_ns3m_node.txt', node)
totcol = 'green!80!blue'
node = end_node(data['44X72'], totcol, percent=True, 
                date=date_tot, offset=offset_tot, full_year=True)
write_txt(text_dir / 'rs_44x72_node.txt', node)

totval = df['44X72'].iloc[-1]
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
shgdp = (totval * 12 / gdp.iloc[-1] * 100)[-1]
datelt = df.index[-1].strftime('%B %Y')
grtxt = inc_dec_percent(totgr)
nsval = df['454'].iloc[-1]
shgdp2 = (nsval * 12 / gdp.iloc[-1] * 100)[-1]
grtxt2 = inc_dec_percent(nsgr)
url = 'https://www.census.gov/retail/index.html'    
text = (f'According to the \href{{{url}}}{{Census Bureau}}, retail '+
        f'and food service sales total \${totval/1000:,.1f} billion '+
        f'in {datelt}, equivalent to {shgdp:.1f} percent of GDP '+
        'on an annualized basis. Over the past year, retail and '+
        f'food service sales {grtxt}, without adjusting for '+
        f'inflation {c_line(totcol)}. Nonstore sales, which '+
        f'include online retailers, have {grtxt2} over the same '+
        f'period, and total \${nsval/1000:,.1f} billion, or '+
        f'{shgdp2:.1f} percent of GDP. ')
write_txt(text_dir / 'marts.txt', text)
print(text)

According to the \href{https://www.census.gov/retail/index.html}{Census Bureau}, retail and food service sales total \$649.8 billion in January 2022, equivalent to 32.5 percent of GDP on an annualized basis. Over the past year, retail and food service sales increased by 13.0 percent, without adjusting for inflation (see {\color{green!80!blue}\textbf{---}}). Nonstore sales, which include online retailers, have increased by 8.4 percent over the same period, and total \$94.3 billion, or 4.7 percent of GDP. 


### By Type

In [4]:
series_dict = {'441': 'Motor Vehicles \& Parts', 
               '442': 'Furniture \& Home Furnishings', 
               '443': 'Electronics \& Appliance', 
               '444': 'Building \& Garden Equipment', 
               '445': 'Food \& Beverage Stores', 
               '446': 'Health \& Personal Care', 
               '447': 'Gasoline Stations', 
               '448': 'Clothing and Accessories', 
               '451': 'Sports/Hobby/Music/Books', 
               '452': 'General Merchandise', 
               '454': 'Nonstore', 
               '722': 'Food Service \& Drinking Places'}

df = pd.read_csv(data_dir / 'rs_raw.csv', index_col='date', 
                 parse_dates=True)[series_dict.keys()]
    
dpi = (pd.read_csv(data_dir / 'nipa20600.csv', 
                   index_col='date', parse_dates=True)
         .loc['1992':, 'A067RC'].rename('VALUE'))
data = (df.multiply(12).divide(dpi, axis=0) * 100).dropna()
results = (pd.concat([data.iloc[-25], data.iloc[-1]], axis=1)
             .sort_values(data.dropna().index[-1], ascending=False))
results.index = results.index.map(series_dict)
ltmonth = dtxt(data.dropna().index[-1])['mon2']
write_txt(text_dir / 'rs_mon1.txt', ltmonth)
prmonth = dtxt(data.dropna().index[-25])['mon2']
write_txt(text_dir / 'rs_mon2.txt', prmonth)
results.round(2).to_csv(data_dir / 'rs_comp.csv', index_label='name')

### Inventories to Sales Ratio (ISRATIO)

In [21]:
# Nominal
url = ('https://api.census.gov/data/timeseries/eits/mtis/?'+
       'get=cell_value,time_slot_id,category_code&'+
       f'key={census_key}&data_type_code=IR&category_code=TOTBUS&'+
       'time=from+1992&for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
        .set_index('date'))['cell_value'].astype('float')

# Real 
years = ','.join(map(str, range(1988, 2023)))
url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&datasetname=NIUnderlyingDetail&'+
        'TableName=U003BU&LineNumber=1&Frequency=M&'+
       f'Year={years}&ResultFormat=json')
r = requests.get(url).json()
data = r['BEAAPI']['Results']['Data']

s1 = pd.read_csv('raw/real_isratio.csv')
s1['Date'] = pd.to_datetime(s1['Date'])
s1 = s1.set_index('Date')['Value'].sort_index()

s2 = pd.Series({pd.to_datetime(i['TimePeriod'], format='%YM%m'): 
               i['DataValue'] for i in data 
                if i['LineNumber'] == '1'}).astype(float)
s2.index.name = 'Date'
s2.name = 'Value'
s2 = s2.sort_index()
result = s1.append(s2)
result.to_csv(data_dir / 'real_isratio.csv', index_label='date')

# Text
ltmon = dtxt(df.index[-1])['mon1']
prmon = dtxt(df.index[-2])['mon1']
prmon12 = dtxt(df.index[-13])['mon1']
ltval = f'{df.iloc[-1]:.2f}'
prval = f'{df.iloc[-2]:.2f}'
prval12 = f'{df.iloc[-13]:.2f}'
pcval = f'{df.loc["2020-02-01"].mean():.2f}'
lt2mon = dtxt(result.index[-1])['mon1']
pr2mon = dtxt(result.index[-13])['mon1']
lt2val = f'{result.iloc[-1]:.2f}'
lt2val2 = f'{result.iloc[-13]:.2f}'
rngval = f'{result.loc["2019"].mean():.2f}'

text = (f'In {ltmon}, the ratio of total business '+
        f'inventories to sales was {ltval}, compared '+
        f'to {prval} in {prmon}, {prval12} in {prmon12}, '+
        f'and {pcval} in February 2020.\n\nThe '+
        'inflation-adjusted version from BEA shows '+
        f'inventories at {lt2val} times sales in '+
        f'{lt2mon}, following a ratio of {df.iloc[-2]:.2f} '+
        f'in {dtxt(result.index[-2])["mon1"]}, '+
        f'and {lt2val2} one year prior, in {pr2mon}. '+
        f'In 2019, real monthly inventories were {rngval} '+
         'times real monthly sales, on average. ')
write_txt(text_dir / 'isratio.txt', text)
print(text)

In December 2021, the ratio of total business inventories to sales was 1.29, compared to 1.25 in November 2021, 1.35 in December 2020, and 1.42 in February 2020.

The inflation-adjusted version from BEA shows inventories at 1.41 times sales in December 2021, following a ratio of 1.25 in November 2021, and 1.45 one year prior, in December 2020. In 2019, real monthly inventories were 1.51 times real monthly sales, on average. 
